This Python notebook is designed to get the success rate of trades for mobile OTC based on transaction data pulled from the PDAX Admin Portal. Criteria for this extracts are (1) Aggregated by day since inception (Feb 03, 2022) and (2) Aggregated by hour in the last month.

In [16]:
# Import necessary packages for data processing

import csv
import pandas as pd
import matplotlib.pyplot as plt
import plotly.express as px
import plotly.graph_objects as go


In [17]:
# Load csv data from admin portal
# select and rename necessary fields

filename = 'data/OTC_Trade_Ledger_04132022.csv'
df = pd.read_csv(filename , usecols = ['Traded Currency','Trade Side','Status','Create Date'])
df = df.rename(columns={"Traded Currency":"traded_currency","Trade Side":"trade_side","Status":"status","Create Date":"create_date"})

In [18]:
# Fix datetime data fields
df['create_date'] = (df['create_date']).astype(str)
df['create_date'] = df['create_date'].str.replace(r'|','',regex=True)
df['create_date'] = pd.to_datetime(df['create_date']) 
df['processDate'] = df['create_date'].dt.strftime('%Y-%m-%d')
df['processTime'] = df['create_date'].dt.strftime('%H:%M:%S')
df['processHour'] = df['create_date'].dt.strftime('%H')
df['processTime'] = (df['processHour']).astype(int)

In [19]:
# applying filters for successful/rejected status and startdate
start_date = '2022-02-03'
stat_filter = ('successful','rejected')
df2 = df.query('processDate >= @start_date')
df_stat_filter = df2.query('status in @stat_filter')
df_stat_filter = df_stat_filter.loc[:, ['traded_currency','trade_side','status','processDate','processTime','processHour']]

In [20]:
# get number of transaction and success_rate for daily aggregation

df3 = df_stat_filter.loc[:, ['traded_currency','trade_side','status','processDate']]
df3['stat_cnt'] =  df3.groupby(['traded_currency','processDate','status'])['status'].transform('count')
df3['token_cnt']  =  df3.groupby(['traded_currency','processDate'])['processDate'].transform('count')
df3['txn_cnt']  =  df3.groupby(['traded_currency','trade_side','status','processDate'])['processDate'].transform('count')

df3['s_rate'] =  df3['stat_cnt'] / df3['token_cnt'] 
df3['s_rate'] = df3['s_rate'].astype(float).map("{:.2%}".format)

df_daily_agg = df3.loc[:, ['traded_currency','trade_side','status','processDate','txn_cnt','s_rate']]
df_daily_agg = df_daily_agg.drop_duplicates(keep='last')

df_daily_agg

,traded_currency,trade_side,status,processDate,txn_cnt,s_rate
33,MATIC,buy,rejected,2022-04-13,2,4.17%
171,DOT,sell,successful,2022-04-13,2,100.00%
381,AXS,sell,rejected,2022-04-13,2,10.00%
495,DOT,buy,successful,2022-04-13,10,100.00%
531,AVAX,buy,successful,2022-04-13,20,100.00%
...,...,...,...,...,...,...
96025,ADA,buy,successful,2022-02-03,150,98.80%
96027,MATIC,sell,successful,2022-02-03,18,98.25%
96029,DOGE,buy,successful,2022-02-03,186,100.00%
96031,SLP,buy,successful,2022-02-03,8,100.00%


In [21]:
# run this to save the 'df_daily_agg' to csv file:
# df_daily_agg.to_csv(r'df_daily_agg.csv', index = False)

In [22]:
# applying filters to get previous month data

prev_month_start_date = '2022-03-01'
prev_month_end_date = '2022-03-31'
df4 = df_stat_filter.query('processDate >= @prev_month_start_date and processDate <= @prev_month_end_date')
df4 = df4.loc[:, ['traded_currency','trade_side','status','processDate','processHour']]


In [23]:
# get number of transaction and success_rate for hourly aggregation of the previous month

df4['stat_cnt'] =  df4.groupby(['traded_currency','processDate','processHour','status'])['status'].transform('count')
df4['token_cnt']  =  df4.groupby(['traded_currency','processDate','processHour'])['processHour'].transform('count')
df4['txn_cnt']  =  df4.groupby(['traded_currency','trade_side','status','processDate','processHour'])['processHour'].transform('count')
df4['s_rate'] =  df4['stat_cnt'] / df4['token_cnt'] 
df4['s_rate'] = df4['s_rate'].astype(float).map("{:.2%}".format)

df_hourly_agg = df4.loc[:, ['traded_currency','trade_side','status','processDate','processHour','txn_cnt','s_rate']]
df_hourly_agg = df_hourly_agg.drop_duplicates(keep='last')
df_hourly_agg


,traded_currency,trade_side,status,processDate,processHour,txn_cnt,s_rate
23437,DOT,sell,successful,2022-03-31,23,2,100.00%
23441,AVAX,buy,successful,2022-03-31,23,2,100.00%
23445,DOGE,buy,successful,2022-03-31,23,8,100.00%
23449,AVAX,sell,successful,2022-03-31,23,4,100.00%
23451,ADA,buy,successful,2022-03-31,23,6,100.00%
...,...,...,...,...,...,...,...
68449,MATIC,sell,successful,2022-03-01,00,6,100.00%
68451,DOT,sell,successful,2022-03-01,00,6,100.00%
68455,AVAX,sell,successful,2022-03-01,00,6,100.00%
68459,SLP,sell,rejected,2022-03-01,00,6,17.65%


In [24]:
# run this to save the 'df_hourly_agg' to csv file:
# df_hourly_agg.to_csv(r'df_hourly_agg.csv', index = False)

Create filters for successful status per trade_side for daily aggregation

In [25]:
#successful status
df_daily_agg_1=df_daily_agg[df_daily_agg["status"]=="successful"]

#successful buy transactions
df_daily_agg_buy=df_daily_agg_1[df_daily_agg_1["trade_side"]=="buy"]
df_daily_agg_buy['s_rate']= df_daily_agg_buy['s_rate'].str.replace(r'%','',regex=True).astype(float)

#successful sell transactions
df_daily_agg_sell=df_daily_agg_1[df_daily_agg_1["trade_side"]=="sell"]
df_daily_agg_sell['s_rate']= df_daily_agg_sell['s_rate'].str.replace(r'%','',regex=True).astype(float)


/var/folders/nl/v30g3wnn7jsfhyymx_pg7jq40000gp/T/ipykernel_30776/3945318639.py:6: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/var/folders/nl/v30g3wnn7jsfhyymx_pg7jq40000gp/T/ipykernel_30776/3945318639.py:10: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [26]:
# Mobile OTC Buy Transactions Daily Success Rate

fig = go.Figure(data=[
    go.Scatter(name='SLP', x=df_daily_agg_buy[df_daily_agg_buy["traded_currency"]=="SLP"]["processDate"], y=df_daily_agg_buy[df_daily_agg_buy["traded_currency"]=="SLP"]["s_rate"], marker_color='RGB(255,148,162)'),
    go.Scatter(name='AXS', x=df_daily_agg_buy[df_daily_agg_buy["traded_currency"]=="AXS"]["processDate"], y=df_daily_agg_buy[df_daily_agg_buy["traded_currency"]=="AXS"]["s_rate"], marker_color='RGB(0,8,105)'),
    go.Scatter(name='ADA', x=df_daily_agg_buy[df_daily_agg_buy["traded_currency"]=="ADA"]["processDate"], y=df_daily_agg_buy[df_daily_agg_buy["traded_currency"]=="ADA"]["s_rate"], marker_color='RGB(53,121,212)'),
    go.Scatter(name='DOGE', x=df_daily_agg_buy[df_daily_agg_buy["traded_currency"]=="DOGE"]["processDate"], y=df_daily_agg_buy[df_daily_agg_buy["traded_currency"]=="DOGE"]["s_rate"], marker_color='RGB(188,161,48)'),
    go.Scatter(name='AVAX', x=df_daily_agg_buy[df_daily_agg_buy["traded_currency"]=="AVAX"]["processDate"], y=df_daily_agg_buy[df_daily_agg_buy["traded_currency"]=="AVAX"]["s_rate"], marker_color='RGB(233,63,64)'),
    go.Scatter(name='MATIC', x=df_daily_agg_buy[df_daily_agg_buy["traded_currency"]=="MATIC"]["processDate"], y=df_daily_agg_buy[df_daily_agg_buy["traded_currency"]=="MATIC"]["s_rate"], marker_color='RGB(131,69,230)'),
    go.Scatter(name='DOT', x=df_daily_agg_buy[df_daily_agg_buy["traded_currency"]=="DOT"]["processDate"], y=df_daily_agg_buy[df_daily_agg_buy["traded_currency"]=="DOT"]["s_rate"], marker_color='RGB(231,0,123)')
])
fig.update_layout(
    title_text="Mobile OTC Buy Transactions Daily Success Rate")
fig.update_layout(yaxis_range=[0,100])
fig.show()

In [27]:
# Mobile OTC Sell Transactions Daily Success Rate
fig = go.Figure(data=[
    go.Scatter(name='SLP', x=df_daily_agg_sell[df_daily_agg_sell["traded_currency"]=="SLP"]["processDate"], y=df_daily_agg_sell[df_daily_agg_sell["traded_currency"]=="SLP"]["s_rate"], marker_color='RGB(255,148,162)'),
    go.Scatter(name='AXS', x=df_daily_agg_sell[df_daily_agg_sell["traded_currency"]=="AXS"]["processDate"], y=df_daily_agg_sell[df_daily_agg_sell["traded_currency"]=="AXS"]["s_rate"], marker_color='RGB(0,8,105)'),
    go.Scatter(name='ADA', x=df_daily_agg_sell[df_daily_agg_sell["traded_currency"]=="ADA"]["processDate"], y=df_daily_agg_sell[df_daily_agg_sell["traded_currency"]=="ADA"]["s_rate"], marker_color='RGB(53,121,212)'),
    go.Scatter(name='DOGE', x=df_daily_agg_sell[df_daily_agg_sell["traded_currency"]=="DOGE"]["processDate"], y=df_daily_agg_sell[df_daily_agg_sell["traded_currency"]=="DOGE"]["s_rate"], marker_color='RGB(188,161,48)'),
    go.Scatter(name='AVAX', x=df_daily_agg_sell[df_daily_agg_sell["traded_currency"]=="AVAX"]["processDate"], y=df_daily_agg_sell[df_daily_agg_sell["traded_currency"]=="AVAX"]["s_rate"], marker_color='RGB(233,63,64)'),
    go.Scatter(name='MATIC', x=df_daily_agg_sell[df_daily_agg_sell["traded_currency"]=="MATIC"]["processDate"], y=df_daily_agg_sell[df_daily_agg_sell["traded_currency"]=="MATIC"]["s_rate"], marker_color='RGB(131,69,230)'),
    go.Scatter(name='DOT', x=df_daily_agg_sell[df_daily_agg_sell["traded_currency"]=="DOT"]["processDate"], y=df_daily_agg_sell[df_daily_agg_sell["traded_currency"]=="DOT"]["s_rate"], marker_color='RGB(231,0,123)')
])
fig.update_layout(
    title_text="Mobile OTC Sell Transactions Daily Success Rate")
fig.update_layout(yaxis_range=[0,100])
fig.show()

Create filters for successful status per trade_side for hourly aggregation

In [28]:
#successful status
df_hourly_agg_1=df_hourly_agg[df_hourly_agg["status"]=="successful"]

#successful buy transactions
df_hourly_agg_buy=df_hourly_agg_1[df_hourly_agg_1["trade_side"]=="buy"]
df_hourly_agg_buy['s_rate']= df_hourly_agg_buy['s_rate'].str.replace(r'%','',regex=True).astype(float)
df_hourly_agg_buy = df_hourly_agg_buy.sort_values(by=['processHour'])

#successful sell transactions
df_hourly_agg_sell=df_hourly_agg_1[df_hourly_agg_1["trade_side"]=="sell"]
df_hourly_agg_sell['s_rate']= df_hourly_agg_sell['s_rate'].str.replace(r'%','',regex=True).astype(float)
df_hourly_agg_sell = df_hourly_agg_buy.sort_values(by=['processHour'])

/var/folders/nl/v30g3wnn7jsfhyymx_pg7jq40000gp/T/ipykernel_30776/3635589353.py:6: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/var/folders/nl/v30g3wnn7jsfhyymx_pg7jq40000gp/T/ipykernel_30776/3635589353.py:11: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [29]:
# Mobile OTC March Buy Transactions Success Rate by Hour
fig = go.Figure(data=[
    go.Box(name='SLP', x=df_hourly_agg_buy[df_hourly_agg_buy["traded_currency"]=="SLP"]["processHour"], y=df_hourly_agg_buy[df_hourly_agg_buy["traded_currency"]=="SLP"]["s_rate"], marker_color='RGB(255,148,162)',boxmean=True),
    go.Box(name='AXS', x=df_hourly_agg_buy[df_hourly_agg_buy["traded_currency"]=="AXS"]["processHour"], y=df_hourly_agg_buy[df_hourly_agg_buy["traded_currency"]=="AXS"]["s_rate"], marker_color='RGB(0,8,105)',boxmean=True),
    go.Box(name='ADA', x=df_hourly_agg_buy[df_hourly_agg_buy["traded_currency"]=="ADA"]["processHour"], y=df_hourly_agg_buy[df_hourly_agg_buy["traded_currency"]=="ADA"]["s_rate"], marker_color='RGB(53,121,212)',boxmean=True),
    go.Box(name='DOGE', x=df_hourly_agg_buy[df_hourly_agg_buy["traded_currency"]=="DOGE"]["processHour"], y=df_hourly_agg_buy[df_hourly_agg_buy["traded_currency"]=="DOGE"]["s_rate"], marker_color='RGB(188,161,48)',boxmean=True),
    go.Box(name='AVAX', x=df_hourly_agg_buy[df_hourly_agg_buy["traded_currency"]=="AVAX"]["processHour"], y=df_hourly_agg_buy[df_hourly_agg_buy["traded_currency"]=="AVAX"]["s_rate"], marker_color='RGB(233,63,64)',boxmean=True),
    go.Box(name='MATIC', x=df_hourly_agg_buy[df_hourly_agg_buy["traded_currency"]=="MATIC"]["processHour"], y=df_hourly_agg_buy[df_hourly_agg_buy["traded_currency"]=="MATIC"]["s_rate"], marker_color='RGB(131,69,230)',boxmean=True),
    go.Box(name='DOT', x=df_hourly_agg_buy[df_hourly_agg_buy["traded_currency"]=="DOT"]["processHour"], y=df_hourly_agg_buy[df_hourly_agg_buy["traded_currency"]=="DOT"]["s_rate"], marker_color='RGB(231,0,123)',boxmean=True)
])
fig.update_layout(
    title_text="Mobile OTC March Buy Transactions Success Rate by Hour")
fig.update_layout(yaxis_range=[0,100])
fig.show()

In [30]:
# Mobile OTC March Sell Transactions Success Rate by Hour

fig = go.Figure(data=[
    go.Box(name='SLP', x=df_hourly_agg_sell[df_hourly_agg_sell["traded_currency"]=="SLP"]["processHour"], y=df_hourly_agg_sell[df_hourly_agg_sell["traded_currency"]=="SLP"]["s_rate"], marker_color='RGB(255,148,162)',boxmean=True),
    go.Box(name='AXS', x=df_hourly_agg_sell[df_hourly_agg_sell["traded_currency"]=="AXS"]["processHour"], y=df_hourly_agg_sell[df_hourly_agg_sell["traded_currency"]=="AXS"]["s_rate"], marker_color='RGB(0,8,105)',boxmean=True),
    go.Box(name='ADA', x=df_hourly_agg_sell[df_hourly_agg_sell["traded_currency"]=="ADA"]["processHour"], y=df_hourly_agg_sell[df_hourly_agg_sell["traded_currency"]=="ADA"]["s_rate"], marker_color='RGB(53,121,212)',boxmean=True),
    go.Box(name='DOGE', x=df_hourly_agg_sell[df_hourly_agg_sell["traded_currency"]=="DOGE"]["processHour"], y=df_hourly_agg_sell[df_hourly_agg_sell["traded_currency"]=="DOGE"]["s_rate"], marker_color='RGB(188,161,48)',boxmean=True),
    go.Box(name='AVAX', x=df_hourly_agg_sell[df_hourly_agg_sell["traded_currency"]=="AVAX"]["processHour"], y=df_hourly_agg_sell[df_hourly_agg_sell["traded_currency"]=="AVAX"]["s_rate"], marker_color='RGB(233,63,64)',boxmean=True),
    go.Box(name='MATIC', x=df_hourly_agg_sell[df_hourly_agg_sell["traded_currency"]=="MATIC"]["processHour"], y=df_hourly_agg_sell[df_hourly_agg_sell["traded_currency"]=="MATIC"]["s_rate"], marker_color='RGB(131,69,230)',boxmean=True),
    go.Box(name='DOT', x=df_hourly_agg_sell[df_hourly_agg_sell["traded_currency"]=="DOT"]["processHour"], y=df_hourly_agg_sell[df_hourly_agg_sell["traded_currency"]=="DOT"]["s_rate"], marker_color='RGB(231,0,123)',boxmean=True)
])
fig.update_layout(
    title_text="Mobile OTC March Sell Transactions Success Rate by Hour")
fig.update_layout(yaxis_range=[0,100])

fig.show()